# CWE Analysis
The purpose of this pipeline is to collect metadata about the CWE. In particular view CWE-1003 as it's contains the weaknesses the NVD use for labeling. However we may also take a look at view CWE-1000 for completeness. 

In [45]:
import requests
import json

def req_CWE(url, jfile = None, printj = True) :
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0'
    }
    try:
        # Make the request
        reply = requests.get(url, headers=headers, timeout=100)
        jreply = reply.json()

        if not printj :
            return jreply
        # Write the JSON response to a file
        if jfile is None : 
            print(json.dump(jreply, indent=4))
        else :
            with open('tmp/' + jfile, 'w') as file:
                json.dump(jreply, file, indent=4)  # Write JSON data to the file with indentation
            print("Data successfully written to 'tmp/" + jfile + "'")
        return jreply
    except Exception as exp:
        # Handle exceptions and write error details to the file
        if jfile is None  :
            print(f"Exception -> {exp}")
            print(f"Request status -> {reply.status_code}")
        else :
            with open('tmp/' + jfile, 'w') as file:
                error_message = {
                    "error": str(exp),
                    "status_code": reply.status_code if 'reply' in locals() else None
                }
                json.dump(error_message, file, indent=4)
            print(f"Exception occurred. Details written to 'tmp/" + jfile + "'")

# temporary
url_view1003 = 'https://cwe-api.mitre.org/api/v1/cwe/view/1003'
url_view1003_des = 'https://cwe-api.mitre.org/api/v1/cwe/74/descendants?view=1003'
url_view1003_children = 'https://cwe-api.mitre.org/api/v1/cwe/74/children?view=1003'
url_CWE_20_74_inf = 'https://cwe-api.mitre.org/api/v1/cwe/weakness/20,74'
#url_CWE_all1003_inf = 'https://cwe-api.mitre.org/api/v1/cwe/weakness?view=1003'
url_leaf1284 = 'https://cwe-api.mitre.org/api/v1/cwe/1284/children?view=1003'
print(len(req_CWE(url_view1003, 'view_CWE-1003.json')['Views'][0]['Members']))
req_CWE(url_view1003_des, 'view_CWE-1003_desc_CWE-74.json')
req_CWE(url_view1003_children, 'view_CWE-1003_children_CWE-74.json')
req_CWE(url_CWE_20_74_inf, 'CWE-20,74_weakness.json')
#req_CWE(url_CWE_all1003_inf, 'CWE-1003_weakness.json')
req_CWE(url_leaf1284, 'url_leaf1284.json')

Data successfully written to 'tmp/view_CWE-1003.json'
37
Data successfully written to 'tmp/view_CWE-1003_desc_CWE-74.json'
Data successfully written to 'tmp/view_CWE-1003_children_CWE-74.json'
Data successfully written to 'tmp/CWE-20,74_weakness.json'
Data successfully written to 'tmp/url_leaf1284.json'


[]

In [ ]:
def rec_expand(view, cweID) : 
    childrenURL = 'https://cwe-api.mitre.org/api/v1/cwe/' + cweID + '/children?view=' + view
    childrenj = req_CWE(childrenURL, printj=False)
    id_list = []
    for child in childrenj : 
        headID = child['ID']
        id_list.append(headID)
        id_list += rec_expand(view, headID)
    
    return id_list

def get_weaknesses_from_view(view, expand = False, save=False) :
    viewURL = 'https://cwe-api.mitre.org/api/v1/cwe/view/' + view
    weaknessesURL = 'https://cwe-api.mitre.org/api/v1/cwe/weakness/'
    id_list = []
    jview = req_CWE(viewURL, printj=False)
    for member in jview['Views'][0]['Members'] :
        headID = member['CweID']
        id_list.append(headID)
        if expand : 
            id_list += rec_expand(view, headID)
    weaknessesURL += ",".join(id_list)
    
    print("Number of weaknesses extracted:\t" + len(id_list))
    #print("resulting API request" + weaknessesURL)

    fsave_name = "view_CWE-" + view + ("_all_weaknesses" if expand else "_head_weaknesses") + ".json"
    jview_complete = req_CWE(weaknessesURL, fsave_name, save)
    return jview_complete

jview1003_complete = get_weaknesses_from_view('1003', True, True)



130
https://cwe-api.mitre.org/api/v1/cwe/weakness/20,1284,129,74,1236,77,78,79,88,89,91,917,94,116,838,119,120,125,787,824,200,203,209,532,269,287,290,294,295,306,307,521,522,640,798,311,312,319,326,327,916,330,331,335,338,345,346,347,352,354,924,362,367,400,770,920,404,401,459,763,772,407,1333,436,444,610,1021,384,601,611,918,662,667,665,1188,908,909,668,134,426,427,428,552,669,212,434,494,565,829,670,617,672,415,416,613,674,776,682,131,190,191,193,369,697,704,681,843,706,178,22,59,732,276,281,754,252,273,476,755,834,835,862,425,863,639,913,1321,470,502,92
Data successfully written to 'tmp/view_CWE-1003_all_weaknesses.json'


In [53]:
print(len(jview1003_complete['Weaknesses']))

130
